In [10]:
import pandas as pd

# Load dataset 
df = pd.read_csv('quality_control_data.csv')
df.head()

,SheetID,BasisWeight,Caliper,Brightness,Opacity
0,SHEET_001,84.483571,4.350778,91.431149,92.513015
1,SHEET_002,81.308678,4.947613,92.243138,93.319457
2,SHEET_003,85.238443,4.994371,94.332205,97.241881
3,SHEET_004,89.615149,4.718634,94.215208,96.831111
4,SHEET_005,80.829233,5.103229,84.489323,94.937295


In [13]:
# Define acceptable spec ranges for each metric
specs = {
    'BasisWeight': (75, 90),
    'Caliper': (0.08, 0.12),
    'Brightness': (85, 100),
    'Opacity': (90, 100)
}

In [14]:
# Function to check if each row meets specifications
def check_spec(row):
    issues = []
    for metric, (min_val, max_val) in specs.items():
        if not (min_val <= row[metric] <= max_val):
            issues.append(metric)
    return issues

In [15]:
# Classify based on number of issues
def classify_sheet(issues):
    if len(issues) == 0:
        return "Premium"
    elif len(issues) == 1:
        return "Standard"
    else:
        return "Rejected"

In [16]:
# Apply spec check and classify
df['Issues'] = df.apply(check_spec, axis=1)
df['Quality'] = df['Issues'].apply(classify_sheet)

# Show classified data
df[['SheetID', 'Quality', 'Issues']].head()

,SheetID,Quality,Issues
0,SHEET_001,Standard,[Caliper]
1,SHEET_002,Standard,[Caliper]
2,SHEET_003,Standard,[Caliper]
3,SHEET_004,Standard,[Caliper]
4,SHEET_005,Rejected,"[Caliper, Brightness]"


In [17]:
# Summary of classifications
summary = df['Quality'].value_counts()
total = len(df)
acceptance_rate = ((summary.get('Premium', 0) + summary.get('Standard', 0)) / total) * 100
rejection_rate = 100 - acceptance_rate

print("===== Quality Summary =====")
print(summary)
print(f"\nAcceptance Rate: {acceptance_rate:.2f}%")
print(f"Rejection Rate: {rejection_rate:.2f}%")

===== Quality Summary =====
Quality
Standard    74
Rejected    26
Name: count, dtype: int64

Acceptance Rate: 74.00%
Rejection Rate: 26.00%


In [18]:
# Extract and count common issues in rejected sheets
all_issues = df[df['Quality'] == 'Rejected']['Issues'].explode()
common_issues = all_issues.value_counts()

print("\n===== Common Quality Issues (Rejected Sheets) =====")
print(common_issues)


===== Common Quality Issues (Rejected Sheets) =====
Issues
Caliper        26
Brightness     11
BasisWeight    11
Opacity         7
Name: count, dtype: int64


In [19]:
print("\n===== Recommendations =====")
for issue, count in common_issues.items():
    print(f"- Investigate frequent issue with '{issue}': occurred in {count} rejected sheets.")


===== Recommendations =====
- Investigate frequent issue with 'Caliper': occurred in 26 rejected sheets.
- Investigate frequent issue with 'Brightness': occurred in 11 rejected sheets.
- Investigate frequent issue with 'BasisWeight': occurred in 11 rejected sheets.
- Investigate frequent issue with 'Opacity': occurred in 7 rejected sheets.
